## Scraping
Here we develop our own scraper for funda.nl. 
This will help us get enough data on Dutch properties to perform some real estate analysis.

### Get all links from a funda.nl city search

In [80]:
#imports for scraping
from selenium import webdriver
import time
import datetime
PATH = "/snap/bin/chromium.chromedriver"

In [81]:
#this method will scrape all house links from any city search on funda.nl
#the links will be saved as city_name_date.txt
def get_all_house_links(url):

    #start driver
    driver = webdriver.Chrome(PATH)
    driver.get(url=url)
    #find last page for loop
    pages = driver.find_element_by_class_name('pagination-pages').text.split()
    last_page = int(pages[len(pages)-1]) + 1
    elems = driver.find_elements_by_xpath("//a[@href]")
    for elem in elems:
        link = elem.get_attribute("href")
        if re.search(city_name + '/huis-', link) or re.search(city_name + '/appartement-', link):
            links.add(link)
    print('Amount of house links scraped:', len(links))
    driver.delete_all_cookies()
    time.sleep(2)

    #loop for all other pages
    for page in range(2, last_page):
        page_url = url+"p"+str(page)
        print(page_url)
        source = driver.get(url=page_url)
        elems = driver.find_elements_by_xpath("//a[@href]")
        for elem in elems:
            link = elem.get_attribute("href")
            if re.search(city_name + '/huis-', link) or re.search(city_name + '/appartement-', link):
                links.add(link)
        print('Amount of house links scraped:', len(links))
        driver.delete_all_cookies()
        time.sleep(2)

    driver.close()
    
    #save as txt file
    dt = datetime.datetime.now()
    day = dt.strftime("%d")
    month = dt.strftime("%m")
    yr = dt.strftime("%y")
    filename = city_name + '_' + day + '_' + month + '_' + yr + '.txt'
    with open(filename,'w') as f:
       f.write(str(links))  # set of numbers & a tuple

In [83]:
url = 'https://www.funda.nl/koop/maastricht/'
links = set()
#get city name from url
city_name = url[26:len(url)-1]
# get_all_house_links(url)
print(city_name)

maastricht


### Create the dataframe of all houses from the links

In [ ]:
import os
driver = webdriver.Chrome(PATH)

df = pd.DataFrame()
df.to_csv('denHaag_real_estate.csv')

# small = links[55:65]

for link in links:
    print(link)

    source = driver.get(url=link)
    try:
        without_header = driver.find_elements_by_class_name("object-kenmerken-list")
        with_header = driver.find_element_by_class_name("object-kenmerken-body").text
        headers = driver.find_elements_by_class_name("object-kenmerken-list-header")
    except:
        without_header = []
        with_header = ''
        headers = []
    
    data = []
    for items in without_header:
        string = items.text
        string = string.split('\n')
        data.extend(string)
        
    with_header = with_header.split('\n')
    
    header_l = []
    for items in headers:
        string = items.text
        string = string.split('\n')
        header_l.extend(string)

    if 'Areas' in data:
        data.remove('Areas')
    #no domain expertise on cadastral data so remove
    if 'Cadastral map' in data:
        index_start = data.index('Cadastral map')-1
        #to find the ending index we need to find the element after 'Exterior space'
        cad_index = header_l.index('Cadastral data')
        try:
            cad_next_header = header_l[cad_index+1]
            with_header_index = with_header.index(cad_next_header)
            index_end = data.index(with_header[with_header_index+1])
            temp_end = data[index_end:len(data)]
            data = data[0:index_start]
            data.extend(temp_end)
        except:
            data = data[0:index_start-1]
    col = data[::2]
    row = data[1::2]
    col.append('url')
    row.append(link)
    res = dict(zip(col, row))
    df = df.append(res, ignore_index=True)
    driver.delete_all_cookies()
    os.remove("denHaag_real_estate.csv")
    time.sleep(2)
    df.to_csv('denHaag_real_estate.csv')

df.head()

Once we managed to get data for a single house we need to find a way to get all the links to all the houses in a search so we can iterate all the house links and create a dataframe

In [74]:
driver = webdriver.Chrome(PATH)
source = driver.get(url=links[0])
# print("Element identified by clazss:",driver.find_element_by_class_name("object-kenmerken-body").text)
print("Views", driver.find_element_by_class_name("object-statistics"))
# time.sleep(5)
# driver.close()
#all the feature information
print("Element identified by class:",driver.find_element_by_class_name("object-kenmerken-body").text)

/tmp/ipykernel_63038/3459951299.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Views <selenium.webdriver.remote.webelement.WebElement (session="42cf530ffec19ff39e0ea690f2e3af9b", element="c3dc58b2-1951-4bed-8b24-7114758cec18")>
Element identified by class: Transfer of ownership
Asking price
€ 653,000 vrij op naam
Asking price per m²
€ 4,599
Listed since
March 10, 2022
Status
Available
Acceptance
Available in consultation
Construction
Kind of house
Single-family home, staggered
Building type
New property
Year of construction
2022
Surface areas and volume
Areas
Living area
142 m²
Volume in cubic meters
560 m³
Layout
Number of rooms
6 rooms (4 bedrooms)
Number of stories
4 stories
Energy
Provisional energy label
A What does this mean?
Exterior space
Garden
Back garden and front garden
Storage space
Shed / storage
Detached wooden storage
Garage
Type of garage
Parking place


/tmp/ipykernel_63038/3459951299.py:4: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Views", driver.find_element_by_class_name("object-statistics"))
/tmp/ipykernel_63038/3459951299.py:8: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Element identified by class:",driver.find_element_by_class_name("object-kenmerken-body").text)


In [75]:
#real estate broker
print("Real estate broker:", driver.find_element_by_class_name("object-contact-aanbieder-link").text)

Real estate broker: Tijs & Cyril Makelaardij


/tmp/ipykernel_63038/2959353010.py:2: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  print("Real estate broker:", driver.find_element_by_class_name("object-contact-aanbieder-link").text)


In [77]:
driver = webdriver.Chrome(PATH)
source = driver.get(url=links[0])
string = driver.find_element_by_class_name("object-kenmerken-body").text
without_header = driver.find_elements_by_class_name("object-kenmerken-list")
headers = driver.find_elements_by_class_name("object-kenmerken-list-header")

data = []
for items in without_header:
    string = items.text
    string = string.split('\n')
    data.extend(string)
    
header_l = []
for items in headers:
    string = items.text
    string = string.split('\n')
    header_l.extend(string)
    
data.remove('Areas')
print(data)
print(header_l)

/tmp/ipykernel_63038/3796400837.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/tmp/ipykernel_63038/3796400837.py:3: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  string = driver.find_element_by_class_name("object-kenmerken-body").text
/tmp/ipykernel_63038/3796400837.py:4: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  without_header = driver.find_elements_by_class_name("object-kenmerken-list")
/tmp/ipykernel_63038/3796400837.py:5: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  headers = driver.find_elements_by_class_name("object-kenmerken-list-header")


['Asking price', '€ 653,000 vrij op naam', 'Asking price per m²', '€ 4,599', 'Listed since', 'March 10, 2022', 'Status', 'Available', 'Acceptance', 'Available in consultation', 'Kind of house', 'Single-family home, staggered', 'Building type', 'New property', 'Year of construction', '2022', 'Living area', '142 m²', 'Volume in cubic meters', '560 m³', 'Number of rooms', '6 rooms (4 bedrooms)', 'Number of stories', '4 stories', 'Provisional energy label', 'A What does this mean?', 'Garden', 'Back garden and front garden', 'Shed / storage', 'Detached wooden storage', 'Type of garage', 'Parking place']
['Transfer of ownership', 'Construction', 'Surface areas and volume', 'Layout', 'Energy', 'Exterior space', 'Storage space', 'Garage']


In [78]:
index_start = data.index('Cadastral map')-1
index_end = data.index('Location')
temp_end = data[index_end:len(data)]
data = data[0:index_start]
data.extend(temp_end)
#no domain expertise on cadastral data so remove
col = data[::2]
row = data[1::2]
print(col)
print(row)

ValueError: 'Cadastral map' is not in list

In [71]:
res = dict(zip(col, row))
print(res)

{'Asking price': '€ 325,000 kosten koper', 'Asking price per m²': '€ 3,963', 'Listed since': '2 weeks', 'Status': 'Available', 'Acceptance': 'Available in consultation', 'VVE (Owners Association) contribution': '€ 370.00 per month', 'Type apartment': 'Apartment with shared street entrance (apartment)', 'Building type': 'Resale property', 'Year of construction': '1962', 'Specific': 'Partly furnished with carpets and curtains', 'Type of roof': 'Flat roof', 'Living area': '82 m²', 'Exterior space attached to the building': '5 m²', 'External storage space': '6 m²', 'Volume in cubic meters': '265 m³', 'Number of rooms': '3 rooms (2 bedrooms)', 'Number of bath rooms': '1 bathroom and 1 separate toilet', 'Bathroom facilities': 'Shower and toilet', 'Number of stories': '10 stories', 'Located at': '5th floor', 'Facilities': 'Electricity', 'Energy label': 'E What does this mean?', 'Insulation': 'No insulation', 'Heating': 'Communal central heating', 'Hot water': 'Electrical boiler', 'Location': 

In [72]:
df = pd.DataFrame()
df = df.append(res, ignore_index=True)
df.head()

,Acceptance,Annual meeting,Asking price,Asking price per m²,Balcony/roof garden,Bathroom facilities,Building insurance,Building type,Energy label,Exterior space attached to the building,...,Reserve fund present,Shed / storage,Specific,Status,Type apartment,Type of parking facilities,Type of roof,VVE (Owners Association) contribution,Volume in cubic meters,Year of construction
0,Available in consultation,Yes,"€ 325,000 kosten koper","€ 3,963",Balcony present,Shower and toilet,Yes,Resale property,E What does this mean?,5 m²,...,Yes,Storage box,Partly furnished with carpets and curtains,Available,Apartment with shared street entrance (apartment),Resident's parking permits,Flat roof,€ 370.00 per month,265 m³,1962


In [79]:
text_file = open('links_maastricht.txt', 'r')
lines = text_file.readline()
links = lines.split(', ')

#remove the curly brackets from the set in first and last line
links[0] = links[0][1:len(links[0])]
links[len(links)-1] = links[len(links)-1][0:len(links[len(links)-1])-1]

for link in links:
    index = links.index(link)
    temp = link[1:len(link)-1]
    links[index] = temp
    
#need to cut links by 360 entries as had error in middle
print(len(links))
print(links[0])

315
https://www.funda.nl/en/koop/maastricht/huis-42636089-b1-bouwnr-49/?navigateSource=resultlist


In [346]:
#merge the three amsterdam csvs together
df1 = pd.read_csv("amsterdam_real_estate.csv")
display(df1)
df2 = pd.read_csv("amsterdam_real_estate1.csv")
display(df2)
df3 = pd.read_csv("amsterdam_real_estate2.csv")
display(df3)

df4 = pd.concat([df1, df2, df3])
display(df4)

df4.to_csv("amsterdam_real_estate_final.csv")


,Unnamed: 0,Acceptance,Asking price,Asking price per m²,Bathroom facilities,Building type,Energy label,Exterior space attached to the building,Facilities,Garden,...,228 m²,373 m³,78 m² (13 meter deep and 6 meter broad),94 m²,Carport,"Double glazing, insulated walls and floor insulation","March 5, 2022",Skylight,Toilet,"Vaillant HR (gas-fired combination boiler from 2007, in ownership)"
0,0,Available in consultation,"€ 1,740,000 kosten koper","€ 6,374","Bath, 2 showers and 2 toilets",Resale property,F What does this mean?,10 m²,Passive ventilation system and TV via cable,Patio/atrium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Available in consultation,"€ 550,000 kosten koper","€ 5,140",Bath and shower,Resale property,NaN,12 m²,Elevator and mechanical ventilation,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Available in consultation,"€ 800,000 kosten koper","€ 6,504",NaN,Resale property,F What does this mean?,4 m²,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,"€ 545,000 vrij op naam","€ 6,124",Shower,New property,A+++ What does this mean?,22 m²,NaN,Sun terrace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,354,Available in consultation,"€ 600,000 kosten koper","€ 5,000",NaN,Resale property,Not available,NaN,Mechanical ventilation and TV via cable,Back garden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,355,Available in consultation,"€ 365,000 kosten koper","€ 4,148",NaN,Resale property,C What does this mean?,11 m²,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,356,Available in consultation,"€ 450,000 kosten koper","€ 6,923",Shower and toilet,Resale property,B What does this mean?,8 m²,Passive ventilation system,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,357,Available in consultation,"€ 495,000 kosten koper","€ 9,900",Shower and toilet,Resale property,D What does this mean?,4 m²,Mechanical ventilation and TV via cable,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Acceptance,Annual meeting,Asking price,Asking price per m²,Balcony/roof garden,Building insurance,Building type,CH boiler,Energy label,...,Storage box,Under offer,11 m²,150 m³,3e woonlaag,50 m²,Box,Douche en toilet,Openbaar parkeren,Side garden
0,0,Available in consultation,Yes,"€ 350,000 kosten koper","€ 7,609",Balcony present,Yes,Resale property,Intergas Kompakt HRE (gas-fired combination bo...,C What does this mean?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Available in consultation,Yes,"€ 290,000 kosten koper","€ 9,355",NaN,Yes,Resale property,"Gas-fired combination boiler from 2017, in own...",C What does this mean?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Available in consultation,Yes,"€ 1,000,000 kosten koper","€ 8,333",Balcony present,Yes,Resale property,"Nefit (gas-fired from 2016, in ownership)",D What does this mean?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Available in consultation,Yes,"€ 325,000 kosten koper","€ 9,848",NaN,Yes,Resale property,"Gas-fired combination boiler from 2010, in own...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Available in consultation,NaN,"€ 695,000 kosten koper","€ 4,427",NaN,NaN,Resale property,NaN,A What does this mean?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,342,Available in consultation,Yes,"€ 450,000 kosten koper","€ 4,412",Balcony present,Yes,Resale property,"Gas-fired combination boiler, to rent",A What does this mean?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,343,Available in consultation,NaN,"€ 324,000 vrij op naam","€ 6,480",Balcony present,NaN,New property,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,344,Available in consultation,NaN,"€ 1,800,000 kosten koper","€ 10,345",Balcony present,NaN,Resale property,"Combination boiler, in ownership",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
345,345,Available in consultation,Yes,"€ 500,000 kosten koper","€ 6,410",Balcony present,Yes,Resale property,"Agpo (gas-fired combination boiler, in ownership)",D What does this mean?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


/home/roobs/environments/DA_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3012: DtypeWarning: Columns (18,19,20,21,22,23,25,26,27,28,30,33,34,35,37,38,79,80,81,88,89,91,93,94,95,96,99,100,103,104,105,106,107,108,109,110,111,112,113,114,115,116,118,119,120,122,123,125,127,128,131,132,133,134,135,136,138,139,140,141,142,143,144,145,146,147,148,150,156,157,158,160,161,162,164,165,166,168,169,170,171,172,173,174,175,177,178,179,180,181,182,183,184,185,186,187,188,189,190,193,194,195,196,197,198,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,248,249,251,252,253,255,257,258,259,261,262,263,264,265,266,268,269,270,271,272,273,275,277,279,280,281,282,283,284,285,286,287,288,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,311,312,313,314,315,316,317,318,319,321,322,323,324,327,330,331,332,335,336,338,339,340,341,342,3

,Unnamed: 0,Acceptance,Asking price,Asking price per m²,Building type,Energy label,Kind of house,Listed since,Living area,Number of rooms,...,41 m²,82 m²,929 m³,Achtertuin,Betaald parkeren en parkeervergunningen,D Wat betekent dit?,"Dakterras aanwezig, balkon aanwezig en frans balkon aanwezig",Gelegen op het westen,"HR-107-ketel (gas gestookt combiketel, eigendom)",Mechanische ventilatie en TV kabel
0,0,Available in consultation,"€ 1,795,000 kosten koper","€ 8,842",Resale property,https://www.funda.nl/en/koop/amsterdam/huis-42...,"Desirable residence/villa, row house",4 months,203 m²,1 room,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,"€ 1,100,000 kosten koper","€ 12,500",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Available in consultation,"€ 290,000 kosten koper","€ 3,816",Resale property,B 1.28 What does this mean?,NaN,6 weeks,76 m²,3 rooms (2 bedrooms),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Available in consultation,"€ 695,000 kosten koper","€ 7,316",Resale property,A What does this mean?,NaN,3 months,95 m²,3 rooms (2 bedrooms),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Available in consultation,"€ 2,150,000 kosten koper","€ 9,773",Resale property,Not required,NaN,2 weeks,220 m²,4 rooms (2 bedrooms),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,1307,Available in consultation,"€ 500,000 kosten koper","€ 5,556",Resale property,A What does this mean?,NaN,4 weeks,90 m²,3 rooms (2 bedrooms),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308,1308,Available in consultation,"€ 325,000 kosten koper","€ 4,452",Resale property,NaN,NaN,2 weeks,73 m²,3 rooms (2 bedrooms),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1309,1309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Perceel,Eigendomssituatie,Aantal kamers,Achtertuin,https://www.funda.nl/en/koop/amsterdam/huis-42...,Isolatie,Soort parkeergelegenheid,Balkon/dakterras,AMSTERDAM U 2992,Energielabel
1310,1310,Available in consultation,"€ 450,000 kosten koper","€ 6,618",Resale property,C What does this mean?,NaN,7 weeks,68 m²,4 rooms (3 bedrooms),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Acceptance,Asking price,Asking price per m²,Bathroom facilities,Building type,Energy label,Exterior space attached to the building,Facilities,Garden,...,41 m²,82 m²,929 m³,Achtertuin,Betaald parkeren en parkeervergunningen,D Wat betekent dit?,"Dakterras aanwezig, balkon aanwezig en frans balkon aanwezig",Gelegen op het westen,"HR-107-ketel (gas gestookt combiketel, eigendom)",Mechanische ventilatie en TV kabel
0,0,Available in consultation,"€ 1,740,000 kosten koper","€ 6,374","Bath, 2 showers and 2 toilets",Resale property,F What does this mean?,10 m²,Passive ventilation system and TV via cable,Patio/atrium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Available in consultation,"€ 550,000 kosten koper","€ 5,140",Bath and shower,Resale property,NaN,12 m²,Elevator and mechanical ventilation,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Available in consultation,"€ 800,000 kosten koper","€ 6,504",NaN,Resale property,F What does this mean?,4 m²,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,"€ 545,000 vrij op naam","€ 6,124",Shower,New property,A+++ What does this mean?,22 m²,NaN,Sun terrace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,1307,Available in consultation,"€ 500,000 kosten koper","€ 5,556",Shower,Resale property,A What does this mean?,8 m²,Electricity,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1308,1308,Available in consultation,"€ 325,000 kosten koper","€ 4,452",Bath,Resale property,NaN,NaN,Mechanical ventilation,Back garden,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1309,1309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Perceel,Eigendomssituatie,Aantal kamers,Achtertuin,https://www.funda.nl/en/koop/amsterdam/huis-42...,Isolatie,Soort parkeergelegenheid,Balkon/dakterras,AMSTERDAM U 2992,Energielabel
1310,1310,Available in consultation,"€ 450,000 kosten koper","€ 6,618",Shower,Resale property,C What does this mean?,4 m²,Passive ventilation system and TV via cable,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [342]:
with pd.option_context('display.max_rows', 5,
                       'display.max_columns', 10,
                       'display.precision', 3,
                       ):
    display(df)

,Acceptance,Asking price,Asking price per m²,Building type,Energy label,...,D Wat betekent dit?,"Dakterras aanwezig, balkon aanwezig en frans balkon aanwezig",Gelegen op het westen,"HR-107-ketel (gas gestookt combiketel, eigendom)",Mechanische ventilatie en TV kabel
0,Available in consultation,"€ 1,795,000 kosten koper","€ 8,842",Resale property,https://www.funda.nl/en/koop/amsterdam/huis-42...,...,NaN,NaN,NaN,NaN,NaN
1,NaN,"€ 1,100,000 kosten koper","€ 12,500",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1310,Available in consultation,"€ 450,000 kosten koper","€ 6,618",Resale property,C What does this mean?,...,NaN,NaN,NaN,NaN,NaN
1311,Available in consultation,"€ 1,800,000 vrij op naam","€ 9,730",New property,NaN,...,NaN,NaN,NaN,NaN,NaN
